In [1]:
import sys
sys.path.insert(0, '../')

In [149]:
from algorithms.CollaborativeFilterItem import CollaborativeFilterItem
from similarity_cython.similarity_cython.CosineSim import Cosine_Similarity
import pandas as pd
import numpy as np
from utility import train_test_split
from utility import read_data
import scipy.sparse as sps
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize

In [73]:
data = pd.read_csv("../Data/train_final.csv",sep='\t')
cfi = CollaborativeFilterItem()

In [74]:
cfi.setup(data)

In [75]:
cfi.urm

<45649x99999 sparse matrix of type '<class 'numpy.int64'>'
	with 1040522 stored elements in Compressed Sparse Row format>

In [14]:
item_item = cosine_similarity(cfi.urm.T.tocsr(), dense_output=False)

In [22]:
def max_n(row_data, row_indices, n):
        i = row_data.argsort()[-n:]
        # i = row_data.argpartition(-n)[-n:]
        top_values = row_data[i]
        top_indices = row_indices[i]  # do the sparse indices matter?
        return top_values, top_indices, i

In [67]:
def pruneTopK(matrix, topK):
    matrix.setdiag(0)
    matrix = matrix.tolil()
    for i in range(0, matrix.shape[0]):
        d,r,_ = max_n(np.array(matrix.data[i]), np.array(matrix.rows[i]),topK)
        matrix.data[i] = d.tolist()
        matrix.rows[i] = r.tolist()
        print("\r%d completes"%i, end='',flush=True)
    print()
    return matrix

In [23]:
item_item_lil = item_item.tolil()

In [29]:
d,r,i=max_n(np.array(item_item_lil.data[1]), np.array(item_item_lil.rows[1]),5)

In [30]:
item_item_lil.data[1] = d.tolist()
item_item_lil.rows[1] = r.tolist()

In [41]:
np.asarray(item_item_lil[1,:].todense()).squeeze().argsort()[-5:]

array([ 2495, 69981,  1702, 36393,     1])

In [37]:
r

array([ 2495, 69981,  1702, 36393,     1])

In [38]:
d

array([ 0.26802813,  0.27854301,  0.29065918,  0.30016681,  1.        ])

In [43]:
for i in range(item_item.shape[0]):
    d,r,_ = max_n(np.array(item_item_lil.data[i]), np.array(item_item_lil.rows[i]),500)
    item_item_lil.data[i] = d.tolist()
    item_item_lil.rows[i] = r.tolist()
    print("\r%d completes"%i, end='',flush=True)
print()

99998 completes


In [50]:
item_item_lil[8,:]

<1x99999 sparse matrix of type '<class 'numpy.float64'>'
	with 500 stored elements in LInked List format>

In [45]:
item_item

<99999x99999 sparse matrix of type '<class 'numpy.float64'>'
	with 60319471 stored elements in Compressed Sparse Row format>

In [134]:
data = sps.csr_matrix([[1,1,1],[1,0,1],[1,1,1],[1,1,1]])

In [135]:
data.todense()

matrix([[1, 1, 1],
        [1, 0, 1],
        [1, 1, 1],
        [1, 1, 1]], dtype=int64)

In [136]:
cos = Cosine_Similarity(data, TopK=1)

In [137]:
cos.compute_similarity().todense()

Similarity item 3 ( 100 % ), 22389.52 item/sec, required time 0.00 min


matrix([[ 0.,  3.,  4.],
        [ 0.,  0.,  0.],
        [ 4.,  0.,  0.]], dtype=float32)

In [146]:
cos = Cosine_Similarity(data, TopK=0)
t = cos.compute_similarity()

Similarity item 3 ( 100 % ), 62914.56 item/sec, required time 0.00 min


In [148]:
t

array([[ 0.,  3.,  4.],
       [ 3.,  0.,  3.],
       [ 4.,  3.,  0.]])

In [151]:
normalize(t, norm='l1', axis=0)

array([[ 0.        ,  0.5       ,  0.57142857],
       [ 0.42857143,  0.        ,  0.42857143],
       [ 0.57142857,  0.5       ,  0.        ]])

In [139]:
result = cosine_similarity(data.T, dense_output=False)

In [140]:
result.setdiag(0)

In [141]:
result.todense()

matrix([[ 0.       ,  0.8660254,  1.       ],
        [ 0.8660254,  0.       ,  0.8660254],
        [ 1.       ,  0.8660254,  0.       ]])

In [133]:
(data.T * data).todense()

matrix([[ 42,  11,  72],
        [ 11,   6,  16],
        [ 72,  16, 132]], dtype=int64)

In [143]:
r = pruneTopK(result, topK=1)

2 completes


In [145]:
r.todense().T

matrix([[ 0.       ,  0.       ,  1.       ],
        [ 0.       ,  0.       ,  0.       ],
        [ 1.       ,  0.8660254,  0.       ]])